In [2]:
!pip install pyTelegramBotAPI pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 4.4 MB/s eta 0:00:00


In [3]:

!ffmpeg -version


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [6]:

TOKEN = '***'


In [8]:
import telebot
import os

bot = telebot.TeleBot(TOKEN)
user_state = {}

SAVE_DIR = 'downloads_original'
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    chat_id = message.chat.id
    if chat_id in user_state:
        del user_state[chat_id]

    welcome_message = (
        "**СНАЧАЛА** отправь мне **ЖЕЛАЕМОЕ ИМЯ** для файла \n"
        "**ЗАТЕМ** отправь сам аудиофайл или голосовое сообщение."
    )
    bot.send_message(chat_id, welcome_message, parse_mode='Markdown')


@bot.message_handler(func=lambda message: message.content_type == 'text')
def handle_text_filename(message):
    chat_id = message.chat.id

    raw_name = message.text.strip()
    sanitized_name = "".join(c for c in raw_name if c.isalnum() or c in (' ', '_', '-')).replace(' ', '_')

    user_state[chat_id] = {'file_name': sanitized_name}

    response = (
        f"Теперь аудиофайл "
    )
    bot.reply_to(message, response, parse_mode='Markdown')



@bot.message_handler(content_types=['audio', 'voice'])
def handle_save_file(message):
    chat_id = message.chat.id

    try:

        custom_file_name_base = user_state[chat_id]['file_name']

        if message.content_type == 'voice':
            file_info = message.voice
            file_extension = "ogg"
        else:
            file_info = message.audio
            file_extension = file_info.file_name.split('.')[-1] if file_info.file_name else "mp3"


        bot.reply_to(message, f"Сохраняю ", parse_mode='Markdown')
        file_path_info = bot.get_file(file_info.file_id)
        downloaded_file_bytes = bot.download_file(file_path_info.file_path)

        final_file_name = f"{custom_file_name_base}.{file_extension}"
        full_save_path = os.path.join(SAVE_DIR, final_file_name)

        with open(full_save_path, 'wb') as new_file:
            new_file.write(downloaded_file_bytes)

        del user_state[chat_id]

        bot.reply_to(message, f"Успех", parse_mode='Markdown')
        print(f"Сохранен файл: {full_save_path}")

    except Exception as e:
        error_message = f" Попробуй /start еще раз. Ошибка: {e}"
        bot.reply_to(message, error_message)

        if chat_id in user_state:
            del user_state[chat_id]
        print(f"Ошибка: {e}")


bot.polling(none_stop=True, interval=0)

Бот запущен...
Сохранен файл: downloads_original/Приготовь_фотографию_в_ночное_время.ogg
Сохранен файл: downloads_original/измени_фон_на_пляж.ogg
Сохранен файл: downloads_original/Смени_закат_на_восход_солнца.ogg
Сохранен файл: downloads_original/Сделай_коттедж_замком.ogg
Сохранен файл: downloads_original/Сделай_его_супергероем.ogg
Сохранен файл: downloads_original/Сделай_как_будто_это_в_Японии.ogg
Сохранен файл: downloads_original/Сделай_из_этого_белого_медведя.ogg
Сохранен файл: downloads_original/Сделай_цифровую_фотографию.ogg
